## Data Manipluation: Halloween Candy

## Overview
Overview:
This project explores the 2017 Halloween Candy Hierarchy survey data to uncover insights about candy preferences and their correlation with demographic information like gender and age. The primary objective of this analysis is to clean and preprocess the dataset for machine learning readiness while exploring patterns in candy joy, despair, and neutrality ratings. Key steps include data cleaning, feature engineering, and encoding to prepare the data for predictive modeling. Finally, the project aims to predict gender based on candy preferences using various machine learning techniques.

## Sections:
1. Import & Data Exploration
2. Data Cleaning & Preprocessing
3. Feature Engineering
4. Data Transformation & Encoding
5. Conclusion

## 1. Import & Data Exploration

In [42]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 20)
import warnings
warnings.filterwarnings("ignore")

In [43]:
# read_csv with iso-8859-1 encoding, there are special characters in the data that Pandas doesn't recognize
candy_full = pd.read_csv('candy.csv', encoding='iso-8859-1')

# copy to new DF so that I can have a copy of the original import if needed
candy = candy_full.copy()

In [44]:
# take a brief look at the dataframe 
candy.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [45]:
# check info about the dataframe 
candy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Columns: 120 entries, Internal ID to Click Coordinates (x, y)
dtypes: float64(4), int64(1), object(115)
memory usage: 2.3+ MB


In [46]:
# display columns by setting the max_cols argument equal to the number of columns in the data set
candy.info(max_cols=120)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 120 columns):
 #    Column                                                                                 Non-Null Count  Dtype  
---   ------                                                                                 --------------  -----  
 0    Internal ID                                                                            2479 non-null   int64  
 1    Q1: GOING OUT?                                                                         2368 non-null   object 
 2    Q2: GENDER                                                                             2437 non-null   object 
 3    Q3: AGE                                                                                2394 non-null   object 
 4    Q4: COUNTRY                                                                            2414 non-null   object 
 5    Q5: STATE, PROVINCE, COUNTY, ETC                                   

In [47]:
# print a list of column names
for col in candy.columns:
    print(col)

Internal ID
Q1: GOING OUT?
Q2: GENDER
Q3: AGE
Q4: COUNTRY
Q5: STATE, PROVINCE, COUNTY, ETC
Q6 | 100 Grand Bar
Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes)
Q6 | Any full-sized candy bar
Q6 | Black Jacks
Q6 | Bonkers (the candy)
Q6 | Bonkers (the board game)
Q6 | Bottle Caps
Q6 | Box'o'Raisins
Q6 | Broken glow stick
Q6 | Butterfinger
Q6 | Cadbury Creme Eggs
Q6 | Candy Corn
Q6 | Candy that is clearly just the stuff given out for free at restaurants
Q6 | Caramellos
Q6 | Cash, or other forms of legal tender
Q6 | Chardonnay
Q6 | Chick-o-Sticks (we donÕt know what that is)
Q6 | Chiclets
Q6 | Coffee Crisp
Q6 | Creepy Religious comics/Chick Tracts
Q6 | Dental paraphenalia
Q6 | Dots
Q6 | Dove Bars
Q6 | Fuzzy Peaches
Q6 | Generic Brand Acetaminophen
Q6 | Glow sticks
Q6 | Goo Goo Clusters
Q6 | Good N' Plenty
Q6 | Gum from baseball cards
Q6 | Gummy Bears straight up
Q6 | Hard Candy
Q6 | Healthy Fruit
Q6 | Heath Bar
Q6 | Hershey's Dark Chocolate
Q6 | HersheyÕ

## 2.  Data Cleaning & Preprocessing

In [48]:
# find sum of fully duplicated rows 
duplicated_rows = candy.duplicated().sum()
duplicated_rows

17

In [49]:
# find how many rows have the same internal ID
duplicated_internal_id = candy.duplicated(subset = ['Internal ID']).sum()
duplicated_internal_id

19

In [50]:
# drop duplicates with the same internal ID
candy.drop_duplicates(subset = ['Internal ID'], inplace=True)
#candy

In [51]:
# drop noisy/irrelevnant columns 
# 'Internal ID' unique identifier for each respondent, doesn't provide any information useful for analysis
# 'Q5: STATE, PROVINCE, COUNTY, ETC' Geographical location may not be directly relevant to candy preferences
# 'Q7: JOY OTHER, Q8: DESPAIR OTHER' collect open-ended responses that are usually sparse and unstructured
# 'Q9: OTHER COMMENTS' free-text comment field that may contain too much noise
# 'Click Coordinates (x, y)' metadata point from the survey about where the user clicked
# 'Unnamed: 113' unnamed column with no meaningful data
columns_to_remove = [
    'Internal ID',
    'Q5: STATE, PROVINCE, COUNTY, ETC',
    'Q7: JOY OTHER',
    'Q8: DESPAIR OTHER',
    'Q9: OTHER COMMENTS',
    'Unnamed: 113',
    'Click Coordinates (x, y)'
]

candy = candy.drop(columns_to_remove, axis=1)
#candy

In [53]:
# check shape of cleaned dataframe 
candy.shape

(2460, 113)

In [54]:
# predicting candy preferences based on gender, take a look at 'Q2: GENDER'
candy['Q2: GENDER'].value_counts()

Q2: GENDER
Male                  1466
Female                 839
I'd rather not say      83
Other                   30
Name: count, dtype: int64

In [55]:
# check null values for 'Q2: GENDER'
candy['Q2: GENDER'].isnull().sum()

42

In [59]:
#remove all rows with a missing value in the 'Q2: GENDER column'
candy.dropna(subset = ['Q2: GENDER'], inplace = True)
candy['Q2: GENDER'].isnull().sum()

0

In [63]:
# Select rows where gender is defined as either "Male" or "Female," 
# since using a binary classification model for this project
male_rows = candy[candy['Q2: GENDER'] == 'Male']
female_rows = candy[candy['Q2: GENDER'] == 'Female']
candy = pd.concat([male_rows,female_rows])
candy

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),...,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1,No,Male,44,USA,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,...,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,NaN,1.0,NaN,NaN
2,NaN,Male,49,USA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,Male,40,us,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,NaN,1.0,NaN,NaN
4,No,Male,23,usa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday,NaN,1.0,NaN,NaN
5,No,Male,NaN,NaN,JOY,DESPAIR,JOY,NaN,NaN,NaN,...,JOY,DESPAIR,DESPAIR,JOY,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,No,Female,40,canada,MEH,DESPAIR,JOY,DESPAIR,MEH,MEH,...,NaN,DESPAIR,DESPAIR,JOY,White and gold,Sunday,NaN,1.0,NaN,NaN
2473,No,Female,26,USA,JOY,MEH,JOY,DESPAIR,MEH,MEH,...,JOY,MEH,MEH,JOY,White and gold,Friday,NaN,1.0,NaN,NaN
2475,No,Female,33,USA,MEH,DESPAIR,JOY,NaN,NaN,NaN,...,JOY,DESPAIR,MEH,JOY,Blue and black,Friday,NaN,1.0,NaN,NaN
2476,No,Female,26,USA,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,...,MEH,DESPAIR,DESPAIR,MEH,Blue and black,Friday,NaN,1.0,NaN,NaN


In [64]:
# check shape of clean dataframe 
candy.shape

(2305, 113)

In [65]:
### check missing values
# check 'Q1: GOING OUT?' to sum of people going out for halloween 
candy['Q1: GOING OUT?'].isnull().sum()

77

In [67]:
# only interested in those DEFINITELY going out for Halloween
# for null values, fill as 'No'
candy['Q1: GOING OUT?'].fillna('No', inplace = True)
#candy

In [69]:
# check that 'Q1: GOING OUT?' only has 'yes' and 'no' values 
candy['Q1: GOING OUT?'].value_counts()

Q1: GOING OUT?
No     2007
Yes     298
Name: count, dtype: int64

In [70]:
# select only the columns related to candy preferences and survey responses
candy_slice = candy.loc[:, 'Q6 | 100 Grand Bar':'Q11: DAY']
candy_slice

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY
1,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday
4,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday
5,JOY,DESPAIR,JOY,NaN,NaN,NaN,MEH,MEH,DESPAIR,JOY,...,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,MEH,DESPAIR,JOY,DESPAIR,MEH,MEH,MEH,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,DESPAIR,DESPAIR,NaN,DESPAIR,DESPAIR,JOY,White and gold,Sunday
2473,JOY,MEH,JOY,DESPAIR,MEH,MEH,MEH,MEH,DESPAIR,JOY,...,JOY,JOY,MEH,MEH,JOY,MEH,MEH,JOY,White and gold,Friday
2475,MEH,DESPAIR,JOY,NaN,NaN,NaN,NaN,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,NaN,NaN,JOY,DESPAIR,MEH,JOY,Blue and black,Friday
2476,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,MEH,DESPAIR,...,MEH,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,MEH,Blue and black,Friday


In [72]:
#fill any missing values in the candy DataFrame for those columns (going from Q6 | 100 Grand Bar to Q11: DAY) 
#with the string NO_ANSWER

columns_to_fill = candy.loc[:, 'Q6 | 100 Grand Bar':'Q11: DAY'].columns
candy[columns_to_fill] = candy[columns_to_fill].fillna('NO_ANSWER')
candy[columns_to_fill]

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY
1,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday
2,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER
3,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday
4,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday
5,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,MEH,MEH,DESPAIR,JOY,...,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,NO_ANSWER,NO_ANSWER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,MEH,DESPAIR,JOY,DESPAIR,MEH,MEH,MEH,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,JOY,White and gold,Sunday
2473,JOY,MEH,JOY,DESPAIR,MEH,MEH,MEH,MEH,DESPAIR,JOY,...,JOY,JOY,MEH,MEH,JOY,MEH,MEH,JOY,White and gold,Friday
2475,MEH,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,JOY,DESPAIR,MEH,JOY,Blue and black,Friday
2476,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,MEH,DESPAIR,...,MEH,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,MEH,Blue and black,Friday


In [76]:
# select specific media-related columns from the dataset
candy[["Q12: MEDIA [Daily Dish]", "Q12: MEDIA [Science]", "Q12: MEDIA [ESPN]", "Q12: MEDIA [Yahoo]"]]


,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1,NaN,1.0,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,1.0,NaN,NaN
4,NaN,1.0,NaN,NaN
5,NaN,1.0,NaN,NaN
...,...,...,...,...
2472,NaN,1.0,NaN,NaN
2473,NaN,1.0,NaN,NaN
2475,NaN,1.0,NaN,NaN
2476,NaN,1.0,NaN,NaN


In [77]:
#  fill the missing Q12 values with 0.0
columns_to_fill = ['Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]', 'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]']

candy[columns_to_fill] = candy[columns_to_fill].fillna(0.0)
candy[columns_to_fill]

,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
5,0.0,1.0,0.0,0.0
...,...,...,...,...
2472,0.0,1.0,0.0,0.0
2473,0.0,1.0,0.0,0.0
2475,0.0,1.0,0.0,0.0
2476,0.0,1.0,0.0,0.0


In [78]:
# check that there are no missing values left for the Q6 to Q12 columns.
candy.loc[:, 'Q6 | 100 Grand Bar':'Q12: MEDIA [Yahoo]'].isnull().sum()

Q6 | 100 Grand Bar                                                                        0
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)    0
Q6 | Any full-sized candy bar                                                             0
Q6 | Black Jacks                                                                          0
Q6 | Bonkers (the candy)                                                                  0
                                                                                         ..
Q11: DAY                                                                                  0
Q12: MEDIA [Daily Dish]                                                                   0
Q12: MEDIA [Science]                                                                      0
Q12: MEDIA [ESPN]                                                                         0
Q12: MEDIA [Yahoo]                                                              

In [80]:
# look at the very messy Q4: COUNTRY column
candy['Q4: COUNTRY'].unique()

array(['USA ', 'USA', 'us', 'usa', nan, 'canada', 'Canada', 'US',
       'Murica', 'uk', 'United States', 'United Kingdom', 'united states',
       'United States ', 'Usa', 'United States of America', 'UAE',
       'England', 'UK', 'canada ', 'United states', 'u.s.a.', 'france',
       'finland', 'unhinged states', 'Mexico', 'Canada ', 'US of A',
       'united states of america', 'Netherlands', 'germany', 'Us',
       'america', 'u.s.', 'U.K. ', 'Costa Rica',
       'The United States of America', 'unite states', 'U.S.', '46',
       "'merica", 'United State', '32', 'australia', 'Can', 'Canae',
       'Trumpistan', 'Korea', 'California', 'U.S.A.',
       'United States of America ', 'Japan', 'South africa', 'Germany',
       'Canada`', 'United Stated', 'France ', 'Australia', 'Uk',
       'Switzerland', 'Denmark', 'New Jersey', 'CANADA', 'Indonesia',
       'United ststes', 'America', 'United Statss', 'Scotland', 'murrika',
       'USA! USA! USA!', 'USAA', 'united States ', 'soviet ca

In [81]:
# check the 'Q4: COUNTRY' number of unique values
candy['Q4: COUNTRY'].nunique()

115

Clean up 'Q4: COUNTRY' to only include four areas: USA, Canada, Europe 
(the continent, not necessarily the European Union), and Other.

In [82]:
# fill the missing values in the Q4: COUNTRY column with Other
candy['Q4: COUNTRY'] = candy['Q4: COUNTRY'].fillna('Other')
candy['Q4: COUNTRY']

1         USA 
2          USA
3           us
4          usa
5        Other
         ...  
2472    canada
2473       USA
2475       USA
2476      USA 
2478       usa
Name: Q4: COUNTRY, Length: 2305, dtype: object

In [83]:
# double check that there are no missing values in the 'Q4: COUNTRY' column
candy['Q4: COUNTRY'].isnull().sum()

0

In [89]:
# double check 'Other' added to 'Q4: COUNTRY'
'Other' in candy['Q4: COUNTRY'].values

True

In [90]:
# combine all Australia entries into 'Other'
australia_entries = candy[candy['Q4: COUNTRY'].str.lower() == 'australia'].index
candy.loc[australia_entries, 'Q4: COUNTRY'] = 'Other'

In [91]:
# combine all United States entries together into USA
usa_variations = [
    'USA ', 'USA', 'us', 'usa', 'Us', 'US', 'Murica', 'United States', 'united states',
    'Usa', 'United States ', 'United staes', 'United States of America', 'United states',
    'u.s.a.', 'United States of America ', 'america', 'U.S.A.', 'unhinged states',
    'united states of america', 'US of A', 'The United States', 'North Carolina ',
    'Unied States', 'U S', 'u.s.', 'The United States of America', 'unite states',
    'U.S.', 'USA? Hard to tell anymore..', "'merica", 'United State', 'United Sates',
    'California', 'Unites States', 'USa', 'I pretend to be from Canada, but I am really from the United States.',
    'Usa ', 'United Stated', 'New Jersey', 'United ststes', 'America', 'United Statss',
    'murrika', 'USA! USA! USA!', 'USAA', 'united States ', 'N. America', 'USSA', 'U.S. ',
    'u s a', 'United Statea', 'united ststes', 'USA USA USA!!!!'
]

candy['Q4: COUNTRY'].replace(to_replace=usa_variations, value='USA', inplace=True)

In [92]:
candy['Q4: COUNTRY'].unique()

array(['USA', 'Other', 'canada', 'Canada', 'uk', 'United Kingdom', 'UAE',
       'England', 'UK', 'canada ', 'france', 'finland', 'Mexico',
       'Canada ', 'Netherlands', 'germany', 'U.K. ', 'Costa Rica', '46',
       '32', 'Can', 'Canae', 'Trumpistan', 'Korea', 'Japan',
       'South africa', 'Germany', 'Canada`', 'France ', 'Uk',
       'Switzerland', 'Denmark', 'CANADA', 'Indonesia', 'Scotland',
       'soviet canuckistan', 'Taiwan', 'China', 'Ireland ', 'hong kong',
       'spain', 'Sweden', 'Hong Kong', 'Narnia',
       'subscribe to dm4uz3 on youtube', 'United kingdom',
       "I don't know anymore", 'Fear and Loathing', '35', 'Europe',
       'Greece', '45', 'France', 'Ireland', 'UK ', 'UD', 'Scotland ',
       'South Korea', 'The Netherlands', 'endland', 'Singapore'],
      dtype=object)

In [94]:
# combine the Canadian entries (both upper and lower case) and label them as CA
canada_variations = [
    'canada', 'Canada', 'canada ', 'Canada ', 'Can', 'Canae', 'Canada`', 'CANADA'
]
candy['Q4: COUNTRY'].replace(to_replace=canada_variations, value='CA', inplace=True)

In [96]:
# combine the European entries and label them as EU
europe_variations = [
    'uk', 'United Kingdom', 'England', 'UK', 'france', 'finland', 'Netherlands', 'germany',
    'Europe', 'U.K. ', 'Greece', 'France', 'Ireland', 'Uk', 'Germany', 'Scotland', 'UK ',
    'Denmark', 'France ', 'Switzerland', 'Scotland ', 'The Netherlands', 'Ireland ', 'spain',
    'Sweden', 'United kingdom'
]
candy['Q4: COUNTRY'].replace(to_replace=europe_variations, value='EU', inplace=True)


In [97]:
# combine the other entries and label them as 'Other'
other_variations = candy['Q4: COUNTRY'].loc[~candy['Q4: COUNTRY'].isin(['USA', 'CA', 'EU'])].unique()
candy['Q4: COUNTRY'].replace(to_replace=other_variations, value='Other', inplace=True)

In [98]:
# double check only have four unique values in the 'Q4: COUNTRY' column: USA, Other, CA, and EU
candy['Q4: COUNTRY'].unique()

array(['USA', 'Other', 'CA', 'EU'], dtype=object)

In [100]:
# look at messy 'Q3: AGE'
candy['Q3: AGE'].unique()

array(['44', '49', '40', '23', nan, '53', '33', '43', '56', '64', '48',
       '45', '37', '25', '34', '35', '38', '58', '50', '47', '16', '52',
       '63', '65', '41', '27', '31', '59', '61', '46', '54', '29', '39',
       '36', '28', '42', '69', '67', '30', '70', '51', '19', 'Old enough',
       '57', '60', '24', '32', '62', 'Many', '72', '?', '21', 'no', '68',
       '20', '55', '13', '45-55', '26', '66', '9', '71', '39.4', '74',
       '12', '15', '8', '75', '7', 'Over 50', '90', '22', '76', '6', '11',
       '77', 'OLD', 'old', '70 1/2', '14', 'MY NAME JEFF', '4',
       '59 on the day after Halloween', '73',
       'I can remember when Java was a cool new language', '10', '18',
       'older than dirt', '17', '5u', 'Enough', 'sixty-nine', 'ancient',
       'old enough', 'your mom', '60+'], dtype=object)

# Put 'Q3: AGE' into the following categorical bins: 
unknown, 17 and under, 18-25, 26-35, 36-45, 46-55, and 56+

Missing values should be 'unknown'

Any value with text is 'unknown'

In [102]:
# replace any non-numeric value with a missing value 
# create True/False index
age_index = candy['Q3: AGE'].str.isnumeric()

# for the index, fill missing values with False
age_index = age_index.fillna(False)

# select 'Q3: AGE' column for only those False values from index and code as missing
candy.loc[~age_index, 'Q3: AGE'] = np.nan

In [103]:
# bin the ages
candy['Q3: AGE'] = pd.to_numeric(candy['Q3: AGE'], errors='coerce')


bins = [-float('inf'), 17, 25, 35, 45, 55, float('inf')]
labels = ['17 and under', '18-25', '26-35', '36-45', '46-55', '56+']


candy['Age_Category'] = pd.cut(candy['Q3: AGE'], bins=bins, labels=labels, right=True)
candy['Age_Category'] = candy['Age_Category'].astype(str)  # Convert to string type
candy['Age_Category'] = candy['Age_Category'].replace('nan', 'unknown')  # Replace NaN with 'unknown'


categories = ['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+']
candy['Age_Category'] = pd.Categorical(candy['Age_Category'], categories=categories, ordered=False)


candy['Q3: AGE'] = candy['Age_Category']


candy = candy.drop('Age_Category', axis=1)



In [104]:
# double check 'Q3: AGE' categories
print(candy['Q3: AGE'].cat.categories)

Index(['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'], dtype='object')


In [105]:
candy['Q3: AGE'].value_counts().sort_index()

Q3: AGE
unknown          60
17 and under     49
18-25            85
26-35           520
36-45           768
46-55           525
56+             298
Name: count, dtype: int64

In [106]:
#check to see if there are any missing values in 'Q3: AGE'
candy['Q3: AGE'].isnull().sum()

0

In [107]:
# reset the index for candy ensuring that it goes from 0 to n-1.
candy.reset_index(drop=True, inplace=True)

# 3. Feature Engineering

Feature engineering is the process of transforming raw data into features that better represent the underlying problem to the predictive models. In this section, I will create a new column called "net_feelies" (calculated by the authors as the total joy count minus the total despair count).

First, I'll narrow down the data to make working with it easier.

In [108]:
# select only the Q6 candy columns (Q6 | 100 Grand Bar through Q6 | York Peppermint Patties) in the data set
candy_reduced = candy.loc[:, 'Q6 | 100 Grand Bar':'Q6 | York Peppermint Patties']
candy_reduced

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties
0,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR
1,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER
2,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR
3,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,JOY,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY
4,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,MEH,MEH,DESPAIR,JOY,...,JOY,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,MEH,DESPAIR,JOY,DESPAIR,MEH,MEH,MEH,DESPAIR,DESPAIR,JOY,...,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,JOY
2301,JOY,MEH,JOY,DESPAIR,MEH,MEH,MEH,MEH,DESPAIR,JOY,...,JOY,JOY,JOY,JOY,MEH,MEH,JOY,MEH,MEH,JOY
2302,MEH,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,JOY,...,MEH,MEH,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,JOY,DESPAIR,MEH,JOY
2303,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,MEH,DESPAIR,...,JOY,JOY,MEH,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,MEH


In [109]:
# create a Series called joy_count that lists total counts for JOY for each column, 
# making sure to keep it in the same order as the columns in the candy_reduced DataFrame

joy_counts = []

for col in candy_reduced.columns:
    joy_counts.append(candy_reduced[col].str.contains('JOY', case=False, na=False).sum())


joy_count = pd.Series(joy_counts, index=candy_reduced.columns, name='joy_count')

print(joy_count)

Q6 | 100 Grand Bar                                                                         828
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)     165
Q6 | Any full-sized candy bar                                                             1477
Q6 | Black Jacks                                                                            87
Q6 | Bonkers (the candy)                                                                   109
                                                                                          ... 
Q6 | Vicodin                                                                               674
Q6 | Whatchamacallit Bars                                                                  802
Q6 | White Bread                                                                            43
Q6 | Whole Wheat anything                                                                  110
Q6 | York Peppermint Patties                      

In [110]:
# create a Series called despair_count that lists the total counts for DESPAIR for each column
despair_counts = []

for col in candy_reduced.columns:
    despair_counts.append(candy_reduced[col].str.contains('DESPAIR', case=False, na=False).sum())

despair_count = pd.Series(despair_counts, index=candy_reduced.columns, name='despair_count')

print(despair_count)

Q6 | 100 Grand Bar                                                                          80
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)    1017
Q6 | Any full-sized candy bar                                                               15
Q6 | Black Jacks                                                                           754
Q6 | Bonkers (the candy)                                                                   467
                                                                                          ... 
Q6 | Vicodin                                                                               679
Q6 | Whatchamacallit Bars                                                                  273
Q6 | White Bread                                                                          1376
Q6 | Whole Wheat anything                                                                 1233
Q6 | York Peppermint Patties                      

In [111]:
# take the transpose of the candy_reduced DataFrame and save this transposed data as candy_reduced_transpose
candy_reduced_transpose = candy_reduced.transpose()
candy_reduced_transpose

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,2304
Q6 | 100 Grand Bar,MEH,NO_ANSWER,MEH,JOY,JOY,NO_ANSWER,JOY,JOY,NO_ANSWER,JOY,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,JOY,MEH,JOY,MEH,MEH,DESPAIR
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,NO_ANSWER,MEH,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR
Q6 | Any full-sized candy bar,JOY,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,JOY,JOY,NO_ANSWER,JOY,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,JOY,JOY,JOY,JOY,JOY,JOY
Q6 | Black Jacks,MEH,NO_ANSWER,MEH,DESPAIR,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,NO_ANSWER,MEH,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR
Q6 | Bonkers (the candy),DESPAIR,NO_ANSWER,MEH,MEH,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,NO_ANSWER,MEH,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,MEH,MEH,MEH,NO_ANSWER,MEH,DESPAIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q6 | Vicodin,DESPAIR,NO_ANSWER,JOY,JOY,DESPAIR,NO_ANSWER,MEH,DESPAIR,NO_ANSWER,MEH,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,JOY,DESPAIR,MEH,NO_ANSWER,JOY,JOY
Q6 | Whatchamacallit Bars,DESPAIR,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,DESPAIR,MEH,NO_ANSWER,MEH,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,MEH,NO_ANSWER,JOY,JOY,MEH,DESPAIR
Q6 | White Bread,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,NO_ANSWER,JOY,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH
Q6 | Whole Wheat anything,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,MEH,DESPAIR,MEH,MEH,DESPAIR,DESPAIR


In [113]:
# add a new column called "joy_count" using the joy_count Series above and a new column called 'despair_count" 
# using the despair_count Series above to the candy_reduced_transpose DataFrame
candy_reduced_transpose['net_feelies'] = candy_reduced_transpose['joy_count'] - candy_reduced_transpose['despair_count']

In [114]:
# select only the joy_count, despair_count, and net_feelies columns from the candy_reduced_transpose DataFrame.
# sort this DataFrame in descending order by net_feelies and save this as candy_net_sorted
selected_columns = ['joy_count', 'despair_count', 'net_feelies']
candy_net = candy_reduced_transpose[selected_columns]


candy_net_sorted = candy_net.sort_values(by='net_feelies', ascending=False)
candy_net_sorted

,joy_count,despair_count,net_feelies
Q6 | Any full-sized candy bar,1477,15,1462
Q6 | ReeseÕs Peanut Butter Cups,1416,88,1328
Q6 | Kit Kat,1367,47,1320
"Q6 | Cash, or other forms of legal tender",1363,63,1300
Q6 | Twix,1339,67,1272
...,...,...,...
Q6 | Dental paraphenalia,84,1356,-1272
Q6 | Real Housewives of Orange County Season 9 Blue-Ray,86,1398,-1312
Q6 | White Bread,43,1376,-1333
Q6 | Gum from baseball cards,43,1386,-1343


## 4. Data Transformation & Encoding

I now want to prepare the candy DataFrame to run a machine learning algorthim to determine if I could predict a person's gender based on what candy they prefer

I will now turn all of the values into numeric values

In [115]:
# make a copy of the candy DataFrame and save as candy_encode
candy_encode = candy.copy()

In [116]:
# For the candy_encode DataFrame, replace any Female values with 0 and any Male values with 1
candy_encode['Q2: GENDER'] = candy_encode['Q2: GENDER'].replace({'Female': 0, 'Male': 1})

In [121]:
# select only the 'Q2: GENDER' column from candy_encode and save this as candy_response
# drop the following columns from the candy_encode DataFrame: 
#Q2: GENDER, Q1: GOING OUT?,Q3: AGE,Q4: COUNTRY,Q10: DRESS,Q11: DAY, Q12: MEDIA [Daily Dish],
# Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]. Save the remaining columns as candy_features
candy_response = candy_encode['Q2: GENDER'].reset_index(drop=True)
columns_to_drop = ['Q2: GENDER', 'Q1: GOING OUT?', 'Q3: AGE', 'Q4: COUNTRY',
                   'Q10: DRESS', 'Q11: DAY', 'Q12: MEDIA [Daily Dish]',
                   'Q12: MEDIA [Science]', 'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]']
candy_features = candy_encode.drop(columns=columns_to_drop).reset_index(drop=True)


In [118]:
# Use Panda's get_dummies() to encode the candy_features data, 
#making sure to set drop_first=True and dtype=int. Save this as candy_features_encoded
candy_features_encoded = pd.get_dummies(candy_features, drop_first=True, dtype=int)
candy_features_encoded

,Q6 | 100 Grand Bar_JOY,Q6 | 100 Grand Bar_MEH,Q6 | 100 Grand Bar_NO_ANSWER,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_JOY,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_MEH,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_NO_ANSWER,Q6 | Any full-sized candy bar_JOY,Q6 | Any full-sized candy bar_MEH,Q6 | Any full-sized candy bar_NO_ANSWER,Q6 | Black Jacks_JOY,...,Q6 | Whatchamacallit Bars_NO_ANSWER,Q6 | White Bread_JOY,Q6 | White Bread_MEH,Q6 | White Bread_NO_ANSWER,Q6 | Whole Wheat anything_JOY,Q6 | Whole Wheat anything_MEH,Q6 | Whole Wheat anything_NO_ANSWER,Q6 | York Peppermint Patties_JOY,Q6 | York Peppermint Patties_MEH,Q6 | York Peppermint Patties_NO_ANSWER
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
2,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2301,1,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,0
2302,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2303,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [119]:
# make sure that the candy, candy_features, candy_response and candy_features_encoded have an index that 
# goes from 0 to n-1
print(candy_features_encoded.index)

RangeIndex(start=0, stop=2305, step=1)


# 5. Conclusion

In this project, I took raw survey data from the 2017 Halloween Candy Hierarchy and cleaned it up so it could be used for machine learning. I removed unnecessary columns, filled in missing information, and organized data for better consistency. I also created a new feature called net_feelies to show which candies were liked the most and which were disliked.

After cleaning the data, I converted all the information into a format that machine learning models can understand. Now, the data is ready to be used to predict gender based on candy preferences.

Overall, I transformed messy survey data into a clean, structured format that’s ready for analysis and predictions. The next step would be to actually build and test a machine learning model to see if candy choices can really predict someone's gender.
